In [1]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails

from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Run
from azureml.core.conda_dependencies import CondaDependencies
import json
import pandas as pd
import matplotlib.pyplot as plt
import os

from os import listdir, getcwd, chdir


In [2]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '5b7e9376-1907-45b5-b8cf-6fde28c54e67'
resource_group = 'mpt_projects'
workspace_name = 'ecm_project'

workspace = Workspace(subscription_id, resource_group, workspace_name)

#dataset = Dataset.get_by_name(workspace, name='age_data')
#dataset.download(target_path='.', overwrite=True)

In [3]:
workbookDir = getcwd()
# print('Current Notebook Dir: ' + workbookDir)
# chdir(workbookDir) # Go to current workbook Dir
# chdir('..')        # Go up one
# print(f'Using current directory for loading data: {getcwd()}')
# workbookDir = getcwd()

In [4]:
# filelist = []
# for file in dataset.to_path():
#     filelist.append(workbookDir + file)
# print(filelist[0:4])

In [5]:
ws = Workspace.from_config()
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print(compute.name, ":", compute.type)

nanceGPUcluster1 : AmlCompute
nance8core : ComputeInstance
nanceGPU : ComputeInstance
nanceGPUcluster2 : AmlCompute
nanceGPUcluster3 : AmlCompute
nanceGPU2 : ComputeInstance


In [6]:
# #env = Environment.from_existing_conda_environment(name='training_environment', conda_environment_name='azureml_py36')
# myenv = Environment(name='azureml_py36')

# newenv = myenv.clone('newenv')
# registered_env = newenv.register(ws)
# registered_env.build_local(ws, useDocker=True)

conda_dep = CondaDependencies()
conda_dep.add_conda_package('python=3.8')

test_env = Environment(name='base')
test_env.docker.base_image = "mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04"

with open('/home/azureuser/cloudfiles/code/Users/nlsschim/diff_predictor/requirements.txt', 'r') as f:
    line = f.readline()
    
    while line != '':        
        if line.startswith('#') or len(line.split()) == 0:
            line = f.readline()
            continue
        
        conda_dep.add_pip_package(line.split()[0])
        line = f.readline()

test_env.python.conda_dependencies=conda_dep

In [7]:
test_env.get_image_details

<bound method Environment.get_image_details of {
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "base",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
              

In [8]:
subscription_id = '5b7e9376-1907-45b5-b8cf-6fde28c54e67'
resource_group  = 'mpt_projects'
workspace_name  = 'ecm_project'
ws = Workspace(subscription_id, resource_group, workspace_name)

experiment = Experiment(workspace=ws, name='age_500_model_iters')

In [9]:
from azureml.core import Experiment, ScriptRunConfig

# Create a script config
script_config = ScriptRunConfig(source_directory='./scripts_folder',
                                script='age_xgboost_multimodel_training.py', 
                                compute_target='nanceGPUcluster2',
                                environment=test_env) 



In [10]:
# submit the experiment
run = experiment.submit(config=script_config)
aml_url = run.get_portal_url()
print(aml_url)

https://ml.azure.com/experiments/age_500_model_iters/runs/age_500_model_iters_1623107792_bc1d9bbe?wsid=/subscriptions/5b7e9376-1907-45b5-b8cf-6fde28c54e67/resourcegroups/mpt_projects/workspaces/ecm_project
